::: {.callout-note}
## Learning Outcomes TODO 
- Define and carry out procedure of PCA
- Understand connection between PCA and SVD
:::

## Dimensionality 

So far, we've used visualizations like rugplots or scatterplots to help us identify clusters in our dataset. Since we humans are 3D beings, we can’t visualize beyond three dimensions, but many datasets come with more than three features. In order to visualize multidimensional data, we can reduce the dataset to lower dimensions through **dimensionality reduction**. 

Previously, we have been working with data tables with rows and columns. These rows and columns corresponded to observations and attributes about said observations. Now, we have to be a bit more clear with our wording to follow the language of linear algebra. 

Suppose we have a dataset of:
- N observations (data points/rows)
- d attributes (features/columns).

In Linear Algebra, we think of data being a collection of vectors. Vectors have a *dimension*, meaning they have some number of unique elements. Row and column now denotes the direction a vector is written (horizontally, like a row, or vertically, like a column):

Linear Algebra views our data as a matrix:
- N row vectors in a d-Dimensions, OR
- d column vectors in an N-Dimensions.

**Dimensionality** of data is a complex topic. Sometimes, it is clear from looking at the number of rows/columns, but other times it is not. 

<center><img src = "images/dataset_dims.png"></center>

For example, the dataset below has 4 columns, but the `Weight (lbs)` column is actually just a linear transformation of the `Weight (kg)` column. Thus, no new information is captured, and the matrix of our dataset has a (column) rank of 3! Despite having 4 columns, we still say that this data is 3-dimensional.


<center><img src = "images/dataset4.png"></center>

Plotting the weight columns together reveals the key visual intuition. While the two columns visually span a 2-d space as a line, the data does not deviate at all from that singular line. This means that one of the weight columns are redundant! Even given the option to cover the whole 2d space, the data below does not. It might as well not have this dimension, which is why we still do not consider the data below to span more than 1 dimension.

<center><img src = "images/dataset3.png" width="400vw"></center>

What happens when there are outliers? Below, we've added one outlier point to the dataset above, and just that is enough toi change the rank of the matrix from 1 to 2 dimensions. However, the data is still very much 1-dimensional. 

<center><img src = "images/dataset3_outlier.png" width="400vw"></center>

As such, dimensionality reduction is generally an *approximation* of the original data that's achieved through projecting the data onto a desired dimension. However, there are many ways to project a dataset onto a lower dimension. How do we choose the best one? 

<center><img src = "images/diff_reductions.png" width="1000vw"></center>

In general, we want the projection that is the best approximation for the original data (the graph on the right). In other words, we want the projection that capture the most variance of the original data. In the next section, we'll see how this is calculated.

## Matrix as Transformation

Consider the matrix multiplication example below:

<center><img src = "images/matmul.png"></center>

* For table 1, each row of the fruits matrix represents one bowl of fruit; the first bowl/row has 2 apples, 2 lemons, and 2 melons.
* For table 2, each column of the dollars matrix represents the cost of fruit at a store; the first store/column charges 2 dollars for an apple, 1 dollar for a lemon, and 4 dollars for a melon.
* The output is the cost of each bowl at each store

In general, there are two ways to interpret matrix multiplication:
1. Row dot column to get each datapoint. In this view, we perform multiple linear operations on the data <center><img src = "images/matmul2.png"></center>
2. Linear transformation of columns <center><img src = "images/matmul3.png"></center>

## Principal Component Analysis (PCA) 

In PCA, our goal is to transform observations from high-dimensional data down to low dimensions (often 2, as most visualizations are 2D) through linear transformations. In other words, we want to find a linear transformation that creates a low-dimension representation which captures as much of the original data’s *total variance* as possible.

<center><img src = "images/PCA_1.png"></center>

We often perform PCA during the Exploratory Data Analysis (EDA) stage of our data science lifecycle when we don't know what model to use; it helps us with 

* Visually identifying clusters of similar observations in high dimensions.
* Removing irrelevant dimensions if we suspect that the dataset is inherently low rank. For example, if the columns are collinear: there are many attributes but only a few mostly determine the rest through linear associations.
* Finding a small basis for representing variations in complex things, e.g., images, genes.
* Reducing the number of dimensions to make some computation cheaper.

<center><img src = "images/pca_example.png"></center>


### PCA Procedure (Overview)

1. Center the data matrix by subtracting the mean of each attribute column.
2. To find the $i$-th principal component $v_i$:
    1. $v$ is a unit vector that linearly combines the attributes.
    2. $v$ gives a one-dimensional projection of the data.
    3. $v$ is chosen to minimize the sum of squared distances between each point and its projection onto $v$.
    4. Choose $v$ such that it is orthogonal to all previous principal components.

$k$ principal components capture the most variance of any $k$-dimensional reduction of the data matrix.

In practice, however, we don’t carry out the procedures in step 2 because it takes too long to compute. Instead, we use singular value decomposition (SVD) to efficiently find all principal components.

## Singular Value Decomposition (SVD)

Singular value decomposition (SVD) is an important concept in linear algebra. Since this class requires a linear algebra course (MATH 54, 56 or EECS 16A) as a pre/co-requisite, we assume you have taken (or are taking) a linear algebra course and will not explain SVD in its entirety. In particular, we will go over:

- Why SVD is a valid decomposition of rectangular matrices
- Why PCA is an application of SVD.

We will not go much into the theory and details of SVD. Instead, we will only cover what is needed for a data science interpretation.

::: {.callout-tip}
### [Linear Algebra] Orthonormality

Orthonormal is a combination of two words: orthogonal and normal.
When we say the columns of a matrix are orthonormal, we say that (1) the columns are all orthogonal to each other (all pairs of columns have a dot product of zero) and (2) all columns are unit vectors (he length of each column vector is 1)
<center><img src = "images/orthonormal.png"></center>


Orthonormal matrices have a few important properties
* **Orthonormal inverse**: If an $m \times n$ matrix $Q$ has orthonormal columns,  $QQ^T= Iₘ$ and $Q^TQ=Iₙ$.
* **Rotation of coordinates**: the linear transformation represented by an orthonormal matrix is often a rotation (and less often a reflection). We can imagine columns of the matrix as where the unit vectors of the original space will land.
:::

::: {.callout-tip}
### [Linear Algebra] Diagnomal Matrices 

**Diagonal matrices** are square matrices with non-zero values on the diagonal axis and zero everywhere else.

Right-multiplied diagonal matrices scale each column up or down by a constant factor. Geometrically, this transformation can be viewed as scaling the coordinate system.

<center><img src = "images/diag_matrix.png"></center>
:::

Singular value decomposition (SVD) describes a matrix $X$'s decomposition into three matrices:
$$ X = U \Sigma V^T $$

Let's break down each of these terms one-by-one

### $U$

* $U$ is an $n$ by $d$ matrix: $U \in \mathbb{R}^{n \times d}$.
* Its columns are **orthonormal**: $\bar{u_i}^T\bar{u_j} = 0$ for all pairs $i, j$, and all vectors $\bar{u_i}$ are unit vectors with length = 1.
* Columns of U are called the **left singular vectors**.
* $UU^T = I_n$ and $U^TU = I_d$.
* We can think of $U$ as a rotation. 

<center><img src = "images/u.png"></center>

### $\Sigma$

* $\Sigma$ is a $d$ by $d$ matrix: $\Sigma \in \mathbb{R}^{d \times d}$.
* The majority of the matrix is zero
* It has $r$ **non-zero** **singular values**, and $r$ is the rank of $X$
* Diagonal values (**singular values** $\sigma_1, \sigma_2, ... \sigma_r$), are ordered from greatest to least $\sigma_1 > \sigma_2 > ... > \sigma_r$
* We can think of $\Sigma$ as a rotation. 

<center><img src = "images/sigma.png"></center>

### $V^T$

* $V^T$ is an $d$ by $d$ matrix: $U \in \mathbb{R}^{d \times d}$.
* Columns of $V$ are orthonormal, so the rows of $V^T$ are orthonormal
* Columns of $V$ are called the **right singular vectors**. 
* $VV^T = V^TV = I_d$
* WE can think of $V$ as a rotation. 

<center><img src = "images/v.png"></center>

### SVD: Geometric Perspective 

<center><img src = "images/svd.png"></center>

We’ve seen that $U$ and $V$ represent rotations, and $\Sigma$ represents scaling. Therefore, SVD says that any matrix can be decomposed into a rotation, then a scaling, and another rotation.
<center><img src = "images/svd_geo.png"></center>

### SVD in `numpy`
For this demo, we'll continue working with our rectangular dataset from before with $n=100$ rows and $d=4$ columns.

In [1]:
#| code-fold: false
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(23) #kallisti

plt.rcParams['figure.figsize'] = (4, 4)
plt.rcParams['figure.dpi'] = 150
sns.set()

rectangle = pd.read_csv("data/rectangle_data.csv")
rectangle.head(5)

,width,height,area,perimeter
0,8,6,48,28
1,2,4,8,12
2,1,3,3,8
3,9,3,27,24
4,9,8,72,34


In `numpy`, the SVD algorith is already written and can be called with `np.linalg.svd` ([documentation](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html)). There are multiple versions of SVD; to get the version that we will follow, we need to set the `full_matrices` parameter to `False`.

In [2]:
U, S, Vt = np.linalg.svd(rectangle, full_matrices = False)

First, let's examine `U`. As we can see, it's dimensions are $n \times d$

In [3]:
U.shape


(100, 4)

The first 5 rows of `U` are shown below

In [4]:
pd.DataFrame(U).head(5)

,0,1,2,3
0,-0.155151,0.064830,-0.029935,0.383675
1,-0.038370,-0.089155,0.062019,-0.544421
2,-0.020357,-0.081138,0.058997,0.053418
3,-0.101519,-0.076203,-0.148160,-0.122119
4,-0.218973,0.206423,0.007274,-0.058722


$\Sigma$ is a little different in `NumPy`. Since the only useful values in the diagonal matrix $\Sigma$ are the singular values on the diagonal axis, only those values are returned and they are stored in an array.

Our `rectangle_data` has a rank of $3$, so we should have 3 non-zero singular values, **sorted from largest to smallest**.

In [5]:
S

array([3.62932568e+02, 6.29904732e+01, 2.56544651e+01, 2.86529477e-15])

While none of the values are 0, notice that the last value is so small ($10^{-15}$) that it's practically $0$, and we can round the values. 

In [6]:
np.round(S)

array([363.,  63.,  26.,   0.])

To get `S` in matrix format, we use `np.diag`.

In [7]:
Sm = np.diag(S)
Sm

array([[3.62932568e+02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 6.29904732e+01, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.56544651e+01, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.86529477e-15]])

Finally, we can see that `Vt` is indeed a $d \times d$ matrix

In [8]:
Vt.shape

(4, 4)

In [9]:
pd.DataFrame(Vt)

,0,1,2,3
0,-0.146436,-0.129942,-8.100201e-01,-0.552756
1,-0.192736,-0.189128,5.863482e-01,-0.763727
2,-0.704957,0.709155,7.951614e-03,0.008396
3,0.666667,0.666667,-5.551115e-17,-0.333333


To check that this SVD is a valid decomposition, we can reverse it and see if it matches our original table (it does yay!).

In [10]:
pd.DataFrame(U @ Sm @ Vt).head(5)

,0,1,2,3
0,8.0,6.0,48.0,28.0
1,2.0,4.0,8.0,12.0
2,1.0,3.0,3.0,8.0
3,9.0,3.0,27.0,24.0
4,9.0,8.0,72.0,34.0


## PCA with SVD

Principle Component Analysis PCA and Singular Value Decomposition can be easily mixed up, especially when you have to keep track of so many acronyms. Here is a quick summary:

PCA: a data science procedure used for dimensionality reduction that uses SVD as one of the steps.

SVD: a linear algebra algorithm that splits a matrix into 3 component parts.


### Limited by Rank

One key fact to remember is that the decomposition are not arbitrary. The *rank* of a matrix limits how small our inner dimensions can be if we want to perfectly recreate our matrix. The proof for this is out of scope.

### Automatic Factorization

Even if we know we have to factorize our matrix using an inner dimension of R, that still leaves a large space of solutions to traverse. What if we have a procedure to automatically factorize a rank R matrix into an R dimensional representation with some transformation matrix?

- Lower dimensional representation avoids redundant features.

- Imagine a 1000 dimensional dataset: If the rank is only 5, it’s much easier to do EDA after this mystery procedure.

What if we wanted a 2-D representation? Its valuable to compress all of the data that is relevant onto as few dimensions as possible in order to plot it efficiently. Some 2D matrices yield better approximations than others. How well can we do?

### Capturing Total Variance

Variability of the data is the most important aspect that we want to maintain. It is the bulk of what we care about when we conduct EDA and modeling. Thus, oversimplifying the data may lead to incorrect analyses. 

Variance for a matrix is defined as the sum of the variances of the columns of a matrix. 

<img src = "images/variance.png" width="700">

One approach is to simply keep the columns with the most variability. However, this leads to capturing of a small amount of variability present in our original dataset. Can we do better than just taking two columns from the original matrix?

<img src="images/approach_1.png" width="700">

It turns out that PCA does a better job at preserving our variances. This is one of the best techniques for dimensionality reductions when it comes to preserving variances. Let's dig into how it works!

<img src="images/approach_2.png" width="700">


### PCA vs. SVD

Principle Component Analysis PCA and Singular Value Decomposition can be easily mixed up, especially when you have to keep track of so many acronyms. Here is a quick summary:

PCA: a data science procedure used for dimensionality reduction that uses SVD as one of the steps.

SVD: a linear algebra algorithm that splits a matrix into 3 component parts.

### PCA Procedure

1. Center the data matrix by subtracting the mean of each attribute column.

2. Use SVD to find all $v_i \in \{1...k\}$, the principal components, which fulfills the following criteria:
    
    - $v_i$ is a unit vector that linearly combines the attributes.
    - $v_i$ gives a one-dimensional projection of the data.
    - $v_i$ is chosen to minimize the sum of squared distances between each point and its projection onto v.
    - $v_i$ is orthogonal to all previous principal components.

references for SVD: [EECS 16B Note 14](https://eecs16b.org/notes/sp23/note14.pdf), [EECS 16B Note 15](https://eecs16b.org/notes/sp23/note15.pdf)

### SVD

Singular value decomposition (SVD) is an important concept in linear algebra.

- We assume you have taken (or are taking) a linear algebra course.
- We will not explain SVD in its entirety—in particular, we will not prove:
    - Why SVD is a valid decomposition of rectangular matrices
    - Why PCA is an application of SVD.
- We know SVD is not covered in EECS 16A, nor Math 54 this semester.

We will not go much into the theory and details of SVD. Instead, we will only cover what is needed for a data science interpretation.

<img src="images/svd.png" width="700">

In NumPy, this algorithm is already written and can be called with `np.linalg.svd`.









## Data Variance and Centering

Formally, the i-th singular value tells us the **component score**, i.e., how much of the data variance is captured by the ith principal component. Supposing the number of data points is $n$:

$$\text{i-th component score} = \frac{(\text{i-th singular value}^2)}{n}$$

Summing up the component scores is equivalent to computing total variance.

**Data Centering**: PCA has a data centering step that precedes any singular value decomposition, where if implemented defines the component score as above.

## Proof of component score (out of scope)

The proof defining component score out of scope for this class, but it is included below for your convenience.

*Setup*: Consider the design matrix $X \in \mathbb{R}^{n \times d}$, where the $j$-th column (corresponding to the $j$-th feature) is $x_j \in \mathbb{R}^n$ and the element in row $i$, column $j$ is $x_{ij}$. Further define $\tilde{X}$ as the **centered** design matrix. The $j$-th column is $\tilde{x}_j \in \mathbb{R}^n$ and the element in row $i$, column $j$ is $\tilde{x}_{ij} = x_{ij} - \bar{x_j}$, where $\bar{x_j}$ is the mean of the $x_j$ column vector from the original $X$. 

*Variance*: Construct the **covariance matrix**: $\frac{1}{n} \tilde{X}^T \tilde{X} \in \mathbb{R}^{d \times d}$. The $j$-th element along the diagonal is the **variance** of the $j$-th column of the original design matrix $X$:

$$\left( \frac{1}{n} \tilde{X}^T \tilde{X} \right)_{jj} = \frac{1}{n} \tilde{x}_j ^T \tilde{x}_j = \frac{1}{n} \sum_{i=i}^n (\tilde{x}_{ij} )^2 = \frac{1}{n} \sum_{i=i}^n (x_{ij} - \bar{x_j})^2$$


*SVD*: Suppose singular value decomposition of the *centered* design matrix $\tilde{X}$ yields $\tilde{X} = U \Sigma V^T$, where $U \in \mathbb{R}^{n \times d}$ and $V \in \mathbb{R}^{d \times d}$ are matrices with orthonormal columns, and $\Sigma \in \mathbb{R}^{d \times d}$ is a diagonal matrix with singular values of $\tilde{X}$.

$$\begin{aligned}
\tilde{X}^T \tilde{X} &= (U \Sigma V^T )^T (U \Sigma V^T) \\
&= V \Sigma U^T U \Sigma V^T  & (\Sigma^T = \Sigma) \\
&= V \Sigma^2 V^T & (U^T U = I) \\
\frac{1}{n} \tilde{X}^T \tilde{X} &= \frac{1}{n} V \Sigma V^T =V \left( \frac{1}{n} \Sigma \right) V^T \\
\frac{1}{n} \tilde{X}^T \tilde{X} V &= V \left( \frac{1}{n} \Sigma \right) V^T V = V \left( \frac{1}{n} \Sigma \right) & \text{(right multiply by }V \rightarrow V^T V = I \text{)} \\
V^T \frac{1}{n} \tilde{X}^T \tilde{X} V &= V^T V \left( \frac{1}{n} \Sigma \right) = \frac{1}{n} \Sigma & \text{(left multiply by }V^T \rightarrow V^T V = I \text{)} \\
\left( \frac{1}{n} \tilde{X}^T \tilde{X} \right)_{jj} &= \frac{1}{n}\sigma_j^2  & \text{(Define }\sigma_j\text{ as the} j\text{-th singular value)} \\
\frac{1}{n} \sigma_j^2 &= \frac{1}{n} \sum_{i=i}^n (x_{ij} - \bar{x_j})^2
\end{aligned}$$

The last line defines the $j$-th component score.
